In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
plt.rcParams["font.family"] = "Arial"

Note: Random seed is weird in Jupyter notebook. Setting them in every cell that uses np.random or random just in case.

In [2]:
np.random.seed(1)
random.seed(1)

## Tool

A hardware rental store has a catalog of 20 different tools to rent, spread across 5 different categories (Painting, Concrete, Plumbing, Woodwork, Yardwork). Each tool has a unique name (e.g. “Paint Tool 1”) and belongs to a specific category; the price per day to rent a tool varies by category. You may decide on the pricing of the rental categories.

Tool is an abstract class whose name and available attributes are inherited by the five different tool types modeled in the concrete derived classes. The derived classes have their own category name and price.

In [3]:
class Tool(ABC):
    def __init__(self, name, available):
        self.name = name
        self.available = available 
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"{self.category}, {self.available})")
    

In [4]:
class PaintingTool(Tool):
    def __init__(self, name, available):
        super().__init__(name, available)
        self.category = "Painting"
        self.price = 1
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")

In [5]:
class ConcreteTool(Tool):
    def __init__(self, name, available):
        super().__init__(name, available)
        self.category = "Concrete"
        self.price = 2
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")

In [6]:
class PlumbingTool(Tool):
    def __init__(self, name, available):
        super().__init__(name, available)
        self.category = "Plumbing"
        self.price = 3
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")

In [7]:
class WoodworkTool(Tool):
    def __init__(self, name, available):
        super().__init__(name, available)
        self.category = "Woodwork"
        self.price = 4
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")

In [8]:
class YardworkTool(Tool):
    def __init__(self, name, available):
        super().__init__(name, available)
        self.category = "Yardwork"
        self.price = 5
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price}, "
                f"{self.category}, {self.available})")

## ToolCollection as inventory

In [9]:
np.random.seed(1)
random.seed(1)

class ToolCollection:
    def __init__(self, num_tools=20):
        self.num_tools = num_tools
        self.tool_categories = ["Painting", "Concrete", "Plumbing", "Woodwork", "Yardwork"]
        self.num_tool_categories = len(self.tool_categories)
        
        self.available_tools = []
        self.rented_tools = []
        
        self.generate_tools()
        
    def generate_tools(self):
        # Generate a list of tool objects of various Tool-derived classes. 
        for tool_i in range(self.num_tools):
            random_idx = np.random.randint(0, self.num_tool_categories)
            name = f"T{tool_i+1:02d}-{self.tool_categories[random_idx]}"
            category = self.tool_categories[random_idx]
            available = True
            
            if (category=="Painting"):
                tool_obj = PaintingTool(name, available)  
            if (category=="Concrete"):
                tool_obj = ConcreteTool(name, available)                
            if (category=="Plumbing"):
                tool_obj = PlumbingTool(name, available)                    
            if (category=="Woodwork"):
                tool_obj = WoodworkTool(name, available)
            if (category=="Yardwork"):
                tool_obj = YardworkTool(name, available)
                
            self.available_tools.append(tool_obj)
    
    def __len__(self):
        return len(self.available_tools)
        
    def rent(self, chosen_tools): 
        # Update available status of a tool when rented out
        for chosen_tool in chosen_tools:
            for tool_i, tool in enumerate(self.available_tools):
                if tool.name == chosen_tool.name:
                    tool.available = False
                    self.rented_tools.append(chosen_tool)
                    del self.available_tools[tool_i]
                    break
    
    def restock(self, tools):
        # Update available status of a tool when restock
        for tool in tools:
            tool.available = True
            self.available_tools.append(tool)
    
    def print_tool_list(self):
        for t in self.available_tools:
            print(t)
            
    def __call__(self):
        return self.available_tools  
    
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for tool in self.available_tools:
            yield tool

## Customers

This store has 10 customers; each customer has a unique name and is associated with one of three types. Casual customers rent one or two tools for one or two nights. Business customers always rent three tools for seven nights. Regular customers will rent one to three tools each time they visit for 3 to 5 nights.

In [10]:
class Customer(ABC):
    def __init__(self, name, num_tool_choices, num_night_choices, num_tools_rented):
        self.name = name
        self.num_tool_choices = num_tool_choices
        self.num_night_choices = num_night_choices
        self.num_tools_rented = num_tools_rented
     
    # Before they rent, a customer can choose a number of tools and nights for that rental
    
    def choose_num_tools(self):
        self.num_tools_wanted = random.sample(self.num_tool_choices, 1)[0]

    def choose_num_nights(self):
        self.num_nights = random.sample(self.num_night_choices, 1)[0]
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"{self.num_tools_wanted}, {self.num_nights}), num_tools_rented={self.num_tools_rented}")        

In [11]:
class CasualCustomer(Customer):
    def __init__(self, name, num_tool_choices, num_night_choices, num_tools_rented):
        super().__init__(name, num_tool_choices, num_night_choices, num_tools_rented)
        self.customer_type = "Casual"

    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, {self.customer_type}, "
                f"{self.num_tools_wanted}, {self.num_nights}), num_tools_rented={self.num_tools_rented}")

In [12]:
class BusinessCustomer(Customer):
    def __init__(self, name, num_tool_choices, num_night_choices, num_tools_rented):
        super().__init__(name, num_tool_choices, num_night_choices, num_tools_rented)
        self.customer_type = "Business"
      
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, {self.customer_type}, "
                f"{self.num_tools_wanted}, {self.num_nights}), num_tools_rented={self.num_tools_rented}")

In [13]:
class RegularCustomer(Customer):
    def __init__(self, name, num_tool_choices, num_night_choices, num_tools_rented):
        super().__init__(name, num_tool_choices, num_night_choices, num_tools_rented)
        self.customer_type = "Regular"
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, {self.customer_type}, "
                f"{self.num_tools_wanted}, {self.num_nights}), num_tools_rented={self.num_tools_rented}")

## CustomerCollection

In [14]:
np.random.seed(1)
random.seed(1)

class CustomerCollection:
    def __init__(self, num_customers=10, num_customer_types=3):
        self.num_customers = num_customers
        self.num_customer_types = num_customer_types
        self.customer_types = ["Casual", "Business", "Regular"]
        self.num_tool_choices = [[1,2], [3], [1,2,3]]
        self.num_night_choices = [[1,2], [7], [3,4,5]]

        self.customer_objects = []
        self.generate_customers()
        
    def generate_customers(self):
        # Generate customer object list
        for customer_i in range(self.num_customers):
            random_idx = np.random.randint(0, self.num_customer_types)
 
            customer_params = {
                "name"               : f"C{customer_i+1:02d}-{self.customer_types[random_idx][0]}", 
                "num_tool_choices"   : self.num_tool_choices[random_idx], 
                "num_night_choices"  : self.num_night_choices[random_idx], 
                "num_tools_rented"   : 0
            }
    
            if (self.customer_types[random_idx]=="Casual"):
                customer_obj = CasualCustomer(**customer_params)
            if (self.customer_types[random_idx]=="Business"): 
                customer_obj = BusinessCustomer(**customer_params) 
            if (self.customer_types[random_idx]=="Regular"):  
                customer_obj = RegularCustomer(**customer_params)            

            self.customer_objects.append(customer_obj)
                
    def get_customers(self):
        # For a given day, create list of customers that might come in to rent
        customer_pool = []
        
        for customer in self.customer_objects:
            if customer.num_tools_rented < 3:  # Only customers with 0, 1, 2 active tools can rent
                customer_pool.append(customer)
                
        for customer in customer_pool:
            customer.choose_num_tools()
            customer.choose_num_nights()
            
        todays_customers = random.sample(customer_pool, np.random.randint(len(customer_pool)))        
        return todays_customers
            
    def return_tool(self, customer_name, num_tools_returned):
        # Update customer number of tools in possession if they return
        for customer in self.customer_objects:
            if customer.name == customer_name:
                customer.num_tools_rented -= num_tools_returned
                break
                
    def print_customer_list(self):
        for c in self.customer_objects:
            print(c)
            
    def __call__(self):
        return self.customer_objects
    
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for customer in self.customer_objects:
            yield customer

## Rental

Each time a customer comes into the store, a Rental is created that will keep track of what tools they rented and how many nights they will keep the tools.

A customer can have more than one active rental. That is, they can show up on day 1
and rent 1 tool for 5 nights. They can then show up on day 2 and rent another tool for 4
nights. As long as they do not have more than 3 tools rented, they are allowed to have
multiple rentals.

In [15]:
class Rental:
    def __init__(self, customer_name, tools_rented, 
                 num_rent_nights, total_price,
                 day_rented, day_due, returned):
        
        self.customer_name = customer_name
        self.tools_rented = tools_rented
        self.num_rent_nights = num_rent_nights
        self.total_price = total_price
        self.day_rented = day_rented
        self.day_due = day_due
        self.returned = returned
        
    def set_returned(self):
        self.returned = True
        
    def __repr__(self):
        return (f"{self.__class__.__name__}, Customer: {self.customer_name}, "
                f"({self.tools_rented}, RentDate: {self.day_rented+1}, "
                f"DueDate: {self.day_due+1}, TotalPrice: ${self.total_price}, RETURNED: {self.returned})")

## RentalCollection

Keep a history of completed and active rentals as they are made over time.

In [16]:
class RentalCollection:
    def __init__(self):
        self.rental_objects = []
        
    def create_rental(self, customer, chosen_tools, total_price, day_i):
        # For a given customer, make rental record
        rental_params = {
                "customer_name"   : customer.name, 
                "tools_rented"    : chosen_tools, 
                "num_rent_nights" : customer.num_nights, 
                "total_price"     : total_price,
                "day_rented"      : day_i, 
                "day_due"         : day_i + customer.num_nights, 
                "returned"        : False
            }
            
        rental = Rental(**rental_params)
        return rental
    
    def append_rental(self, rental):
        self.rental_objects.append(rental)
            
    def __call__(self):
        return self.rental_objects
    
    def __contains__(self, other):
        pass
    
    def __iter__(self):
        for rental in self.rental_objects:
            yield rental

## Store

The store keeps track of the existing rentals along with the current inventory of the store. As such, when it has zero rentals, there will be 20 tools in its inventory. When it has zero tools in its inventory, it will have multiple rentals that between them account for all 20 tools.

This is a facade interface that the main() program uses to simulate checking inventory, return processing, tool renting, and sorting completed and active rentals in a given day. 

StoreFacadeAbstract is an abstract class that simulates an interface, with only method declarations but no implementation. StoreFacade then inherits from StoreFacadeAbstract and implements the required methods. 

In [17]:
class StoreFacadeAbstract(ABC):
    
    @property
    @abstractmethod
    def tools(self):
        pass
    
    @property
    @abstractmethod
    def rental_collection(self):
        pass
    
    @property
    @abstractmethod
    def customers(self):
        pass
    
    @abstractmethod
    def get_num_tools_available(self):
        pass

    @abstractmethod
    def update_money(self): 
        pass
     
    @abstractmethod
    def update_customers(self): 
        pass
    
    @abstractmethod    
    def process_returns(self): 
        pass
    
    @abstractmethod
    def rent_tools(self): 
        pass
    
    @abstractmethod
    def retrieve_rental_records(self):
        pass

In [18]:
class StoreFacade(StoreFacadeAbstract):
    def __init__(self):
        self._tools = ToolCollection()               # Inventory
        self._rental_collection = RentalCollection() # Rentals
        self._customers = CustomerCollection()
        
        self._total_money = 0
        self._num_avail = 20
        
        self.active_rentals = []
        self.completed_rentals = []
  
    @property
    def tools(self):
        return self._tools
    
    @property
    def rental_collection(self):
        return self._rental_collection
    
    @property
    def customers(self):
        return self._customers
    
    @property
    def total_money(self):
        return self._total_money

    @total_money.setter
    def total_money(self, new_total_money):
        self._total_money = new_total_money
    
    @property
    def num_avail(self):
        return self._num_avail
    
    @num_avail.setter
    def num_avail(self, new_val):
         self._num_avail = new_val
    
    def get_num_tools_available(self):
        self.num_avail = len(self.tools)
        return self.num_avail
    
    def get_available_tools(self):
        return self.tools()
    
    def update_money(self, today_money): 
        self.total_money += today_money
        
    def update_customers(self, customer): 
        customer.num_tools_rented += customer.num_tools_wanted
        
    def process_returns(self, day_i): 
        for rental_receipt in self.rental_collection.rental_objects:
            if rental_receipt.day_due == day_i:
                rental_receipt.set_returned()
                self.tools.restock(rental_receipt.tools_rented)
                self.customers.return_tool(rental_receipt.customer_name, len(rental_receipt.tools_rented))                
                
    def rent_tools(self, day_i): 
        todays_customers = self.customers.get_customers()
        
        todays_customer_names = []
        for customer_i, customer in enumerate(todays_customers):
                        
            if customer.num_tools_wanted <= self.num_avail and customer.num_tools_wanted + customer.num_tools_rented <= 3:
                
                available_tools = self.get_available_tools()
                chosen_tools = random.sample(available_tools, customer.num_tools_wanted)
                self.tools.rent(chosen_tools)
                total_price = sum([tool.price for tool in chosen_tools] * customer.num_nights)

                # Make a Rental record for this customer & update rental records
                rental = self.rental_collection.create_rental(customer, chosen_tools, total_price, day_i)
                self.rental_collection.append_rental(rental)
                                
                # Update total earning
                self.update_money(rental.total_price)

                # Each day, update num_tools_rented for Customer objects that rent
                self.update_customers(customer)

                # Update number of tools in inventory
                self.num_avail = self.get_num_tools_available()
                
    def retrieve_rental_records(self):
        for rental in self.rental_collection():
            if rental.returned:
                self.completed_rentals.append(rental)
            else:
                self.active_rentals.append(rental)
            
        rental_history = {"Active rentals": self.active_rentals,
                          "Completed rentals": self.completed_rentals}
        return rental_history
            
    

## Report 

At the end of the simulation, the program will produce a report that includes the following information:
- the number of tools currently in the store along with a list of their names
- the amount of money the store made during the 35 days (including any rentals
that occurred on the 35th day)
- a list of all the completed rentals including which tools were rented by which
customer for how many days along with the total amount of that rental
- a list of all the active rentals that includes all of the information listed in the
previous bullet

In [19]:
class Report:
    def __init__(self, tools, total_money, rental_history):
        self.tools = tools
        self.total_money = total_money
        self.rental_history = rental_history

    def format_rental(self, rental_type):
        report = ""
        for rental_i, rental in enumerate(rental_type):
            customer_name = rental.customer_name
            tools_rented = ", ".join([tool.name for tool in rental.tools_rented])
            rental_length = rental.num_rent_nights
            rental_cost = rental.total_price
            rental_report_1 = f"\t{rental_i+1}. Customer: {customer_name}" 
            rental_report_2 = rental_report_1 + f"\n\tTools rented: {tools_rented}"
            rental_report_3 = rental_report_2 + f"\n\tRental length: {rental_length} days" 
            rental_report_4 = rental_report_3 + f"\n\tCost: ${rental_cost}\n\n"
            report += rental_report_4
        return report

    def write_report(self):
        tool_names = ", ".join([tool.name for tool in self.tools])
        num_tools = len(self.tools)
        
        active_rentals = self.format_rental(self.rental_history["Active rentals"])
        completed_rentals = self.format_rental(self.rental_history["Completed rentals"])

        print(f"---------------- REPORT ----------------")
        print(f"TOTAL EARNING: ${self.total_money} \n")
        print(f"NUMBER OF TOOLS CURRENTLY IN STORE: {num_tools} \n")
        print(f"TOOLS CURRENT IN STORE: {tool_names} \n")
        print(f"ACTIVE RENTALS: ")
        print(f"{active_rentals}")
        print(f"COMPLETED RENTALS: ")
        print(f"{completed_rentals}")

## Main

In [20]:
np.random.seed(1)
random.seed(1)

def main():
    num_days = 35
    print_simulation = False

    # Instantiate StoreFacade object
    store = StoreFacade()

    for day_i in range(num_days):

        #------- Rental returns
        store.process_returns(day_i)

        #------- Open store, check inventory   
        num_tools_available_in_store = store.get_num_tools_available()

        #------- Customers come in if store isn't empty
        if num_tools_available_in_store != 0:
            store.rent_tools(day_i)   

        if print_simulation:
            print(f"---------------- DAY {day_i+1} ----------------")
            print(f"Number of tools available = {num_tools_available_in_store} \n")
            print(f"Tools available today: {store.get_available_tools()}\n")

    # Retrieve data for report
    money_earned = store.total_money
    tools_in_store = store.get_available_tools()
    rental_history = store.retrieve_rental_records()

    # Report
    report = Report(tools_in_store, money_earned, rental_history)
    report.write_report()

if __name__ == "__main__":
    main()

---------------- REPORT ----------------
TOTAL EARNING: $2182 

NUMBER OF TOOLS CURRENTLY IN STORE: 2 

TOOLS CURRENT IN STORE: T06-Painting, T10-Yardwork 

ACTIVE RENTALS: 
	1. Customer: C10-B
	Tools rented: T16-Woodwork, T14-Plumbing, T12-Plumbing
	Rental length: 7 days
	Cost: $70

	2. Customer: C03-B
	Tools rented: T05-Woodwork, T15-Yardwork, T08-Concrete
	Rental length: 7 days
	Cost: $77

	3. Customer: C09-B
	Tools rented: T07-Painting, T13-Yardwork, T04-Concrete
	Rental length: 7 days
	Cost: $56

	4. Customer: C06-R
	Tools rented: T11-Concrete
	Rental length: 5 days
	Cost: $10

	5. Customer: C08-B
	Tools rented: T01-Woodwork, T02-Yardwork, T19-Yardwork
	Rental length: 7 days
	Cost: $98

	6. Customer: C05-C
	Tools rented: T17-Yardwork, T09-Yardwork
	Rental length: 2 days
	Cost: $20

	7. Customer: C07-B
	Tools rented: T20-Plumbing, T03-Painting, T18-Plumbing
	Rental length: 7 days
	Cost: $49


COMPLETED RENTALS: 
	1. Customer: C06-R
	Tools rented: T01-Woodwork
	Rental length: 4 days